Regridded sea ice data filtered to retain only the latitudinal bands where concentrations decrease consistently from north to south

In [ ]:
import numpy as np
import netCDF4
import calendar
from datetime import timedelta
from datetime import datetime
import xarray as xr
import pandas as pd

In [2]:
#Isolate some regional subset of regridded sea ice data
filename_si_nh = 'example_regridded_sea_ice_filenc'
f_si_nh = xr.open_dataset(filename_si_nh)
f_si_nh['time'] = xr.decode_cf(f_si_nh)['time']
f_si_nh = f_si_nh.sel(time=slice('2007', '2014'))
si_nh = f_si_nh.siconc
si_nh = si_nh.sel(lat=slice(60, 90))

# Initialize an empty list to store valid longitudes for each time step
all_strictly_increasing_lons = []

# Time step is months (2007-2014 = 8 years = 96 months) 
for time_step in range(96):
    # Select the time step
    first_time_step = si_nh.isel(time=time_step)

    # Initialize a list to store valid longitudes for this time step
    strictly_increasing_lons = []

    # Iterate over each longitude in the dataset
    for lon in first_time_step.lon.values:
        prime_meridian_si = first_time_step.sel(lon=lon, method="nearest")  # Select nearest longitude

        # Skip if the entire longitude slice is NaN
        if prime_meridian_si.isnull().all():
            continue

        # Calculate the differences in sea ice concentration along the latitude axis
        differences = prime_meridian_si.diff(dim='lat')
        values = prime_meridian_si.values  # Extract the array of sea ice concentrations

        # Check for negative differences, but ignore cases where:
        # - Both the current and previous values are >= 95
        # - Either the current or previous value is < 1
        # There tends to be small deviations both positive and negative in concentration close to values of 0% and 100% which will eliminate an entire
        # latitude band unnecessarily
        errors = [
            i for i in range(len(differences))
            if differences[i] < 0 and not (
                (values[i] >= 90 and values[i + 1] >= 90) or
                (values[i] < 5 or values[i + 1] < 5)
            )
        ]
        if errors:
            print(f"Sea ice concentrations are not strictly increasing at longitude {lon} during time step {f_si_nh.time[time_step].values}.")
        else:
            print(f"Sea ice concentrations are strictly increasing at longitude {lon} during time step {f_si_nh.time[time_step].values}.")

        # Add to valid longitudes if no errors found
        if not errors:
            strictly_increasing_lons.append(lon)

    # Store the valid longitudes for the current time step
    all_strictly_increasing_lons.append(strictly_increasing_lons)

# Initialize an empty list to store the filtered sea ice data for each time step
filtered_data = []

# Loop over the 8 years to filter and save valid and invalid longitudes
for time_step in range(96):
    # Select the time step
    first_time_step = si_nh.isel(time=time_step).copy()

    # Set values to NaN for longitudes that do not meet the strictly increasing condition
    for lon in si_nh.lon.values:
        if lon not in all_strictly_increasing_lons[time_step]:
            first_time_step.loc[{'lon': lon}] = np.nan  # Set to NaN for invalid longitudes

    # Append the filtered data for this time step
    filtered_data.append(first_time_step)

# Concatenate the filtered data for all time steps along the 'time' dimension
filtered_sea_ice_data = xr.concat(filtered_data, dim='time')

# Create a new xarray Dataset for the filtered sea ice data (without interpolation)
sea_ice_dataset = xr.Dataset(
    {
        'sea_ice_concentration': (['time', 'lat', 'lon'], filtered_sea_ice_data.values)
    },
    coords={
        'lat': filtered_sea_ice_data.lat.values,
        'lon': filtered_sea_ice_data.lon.values,
        'time': filtered_sea_ice_data.time.values
    }
)

output_filename = 'example_filtered_sea_ice.nc'

sea_ice_dataset.to_netcdf(output_filename)

print(f"Filtered sea ice data saved to {output_filename}")

NameError: name 'xr' is not defined